In [3]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import requests
import shutil
import bs4
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.wait import WebDriverWait
import pyautogui
import cv2
import time

In [4]:
#import excel that contains tiktok link
df_link = pd.read_excel('link tiktok.xlsx')

In [5]:
#FUNTIONS

#download image function
def down(image, xpath):
    #get link image
    captcha = driver.find_element (By.XPATH, xpath)
    src = captcha.get_attribute('src')

    #download image
    response = requests.get(src, stream=True)
    with open(image, 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)
    del response

#CONTOURS FUNCTION
def bin_img(image):
    img = cv2.imread(image)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    binary_img = cv2.threshold(img_gray, 242, 255, cv2.THRESH_BINARY_INV)[1]

    return binary_img

In [6]:
#empty list
links = []
title = []
username = []
likes = []
views = []
comment = []
share = []

#get url from dataframe
for i in range(len(df_link.iloc[:,0])):
    #open driver chrome
    PATH = "C:\chromedriver_win32\chromedriver.exe"
    driver = webdriver.Chrome(PATH)

    #get tiktok link
    link = df_link.iloc[i,0]

    #open and maximize window
    driver.get(link)
    max = driver.maximize_window()

    #wait until captcha window pop up
    try:
        WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.XPATH, "//*[@id='captcha-verify-image']")))
        pass
    except TimeoutException:
        links.append(link)
        title.append('skip')
        username.append('skip')
        likes.append('skip')
        views.append('skip')
        comment.append('skip')
        share.append('skip')
        continue

    #click zoom in to 125x
    pyautogui.moveTo(1898, 57, duration=0.5)
    pyautogui.click()
    time.sleep(0.5)
    pyautogui.moveTo(1836, 238, duration=0.5)
    pyautogui.doubleClick()

    #CAPTCHA SOLVER
    #download captcha image
    down('captcha_img.jpeg', "//*[@id='captcha-verify-image']")
    #download piece of captcha image
    down('captcha_piece.jpeg', "//*[@id='tiktok-verify-ele']/div/div[2]/img[2]")

    #read img
    img = cv2.imread('captcha_img.jpeg')
    img_piece = cv2.imread('captcha_piece.jpeg')

    capt = bin_img('captcha_img.jpeg')
    piece = bin_img('captcha_piece.jpeg')

    #matching shape
    match = cv2.matchTemplate(capt, piece, cv2.TM_CCOEFF_NORMED)

    #check x, y in maximum value
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(match)

    #width and height
    w = piece.shape[1]
    h = piece.shape[0]

    time.sleep(2)

    #move mouse to slider button
    pyautogui.moveTo(782, 722, duration=1.5)
    #drag mouse to the right position
    pyautogui.dragRel((0.78*max_loc[0]), 0, button='left', duration=2)

    #wait until captcha pop up dissapear
    try:
        WebDriverWait(driver, 15).until_not(EC.visibility_of_element_located((By.XPATH, "//*[@id='captcha-verify-image']")))
        pass
    except TimeoutException:
        links.append(link)
        title.append('skip')
        username.append('skip')
        likes.append('skip')
        views.append('skip')
        comment.append('skip')
        share.append('skip')
        continue

    #exit login bar
    ext_login_bar = driver.find_element(By.XPATH, '//*[@id="login-modal"]/div[2]')
    ext_login_bar.click()

    time.sleep(1)

    #start and pause the video
    pyautogui.moveTo(367, 995, duration=1)
    pyautogui.click(button='left')
    time.sleep(2)
    pyautogui.click(button='left')

    time.sleep(1)

    #ADD USERNAME, LIKES, COMMENT, SHARE
    #find username
    xpath_username= '//*[@id="main-content-video_detail"]/div/div[2]/div[1]/div[2]/div/a[2]/span[1]'
    link_username = driver.find_element(By.XPATH, xpath_username)
    get_link_username = link_username.text
    #find likes
    xpath_likes = '//*[@id="main-content-video_detail"]/div/div[2]/div[1]/div[1]/div[1]/div[4]/div/button[1]/strong'
    link_likes = driver.find_element(By.XPATH, xpath_likes)
    get_link_likes = link_likes.text
    #find comment
    xpath_comment = '//*[@id="main-content-video_detail"]/div/div[2]/div[1]/div[1]/div[1]/div[4]/div/button[2]/strong'
    link_comment = driver.find_element(By.XPATH, xpath_comment)
    get_link_comment = link_comment.text
    #find share
    xpath_share = '//*[@id="main-content-video_detail"]/div/div[2]/div[1]/div[1]/div[1]/div[4]/div/button[3]/strong'
    link_share = driver.find_element(By.XPATH, xpath_share)
    get_link_share = link_share.text

    #adding info to list
    username.append(get_link_username)
    likes.append(get_link_likes)
    comment.append(get_link_comment)
    share.append(get_link_share)

    #open profile
    profile = driver.find_element(By.XPATH, '//*[@id="main-content-video_detail"]/div/div[2]/div[1]/div[2]/div/a[2]/span[1]')
    profile.click()

    time.sleep(2)

    #scroll to the bottom 3 times
    for _ in range(3):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        time.sleep(3)

    #take the html source
    page_html = driver.page_source
    page_soup = bs4.BeautifulSoup(page_html, 'html.parser')
    #find all content
    content = page_soup.find_all('div', {'class':"tiktok-x6y88p-DivItemContainerV2 e19c29qe7"})
    len_content = len(content)

    #loop in the profile to find the same url
    for j in range(1, len_content+1):
        #find url
        xpath_url = f'//*[@id="main-content-others_homepage"]/div/div[2]/div[2]/div/div[{j}]/div[1]/div/div/a'
        try:
            link_url = driver.find_element(By.XPATH, xpath_url)
            get_link_url = link_url.get_attribute('href')
        except:
            links.append(link)
            title.append('skip')
            views.append('skip')
            break

        if get_link_url != link:
            continue
        
        #find views
        xpath_views = f'//*[@id="main-content-others_homepage"]/div/div[2]/div[2]/div/div[{j}]/div[1]/div/div/a/div/div[2]/strong'
        try:
            link_views = driver.find_element(By.XPATH, xpath_views)
            get_link_views = link_views.text
        except:
            get_link_views = 'skip'

        #find title
        xpath_title = f'//*[@id="main-content-others_homepage"]/div/div[2]/div[2]/div/div[{j}]/div[2]/a'
        try:
            link_title = driver.find_element(By.XPATH, xpath_title)
            get_link_title = link_title.get_attribute('title')
        except:
            get_link_title = 'skip'

        #adding info
        links.append(get_link_url)
        title.append(get_link_title)
        views.append(get_link_views)

        break
    
    #exit chrome
    driver.quit()
    time.sleep(3)

C:\Users\user\AppData\Local\Temp\ipykernel_10260\121498113.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=113.0.5672.127)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00978893+48451]
	(No symbol) [0x0090B8A1]
	(No symbol) [0x00815058]
	(No symbol) [0x007FD073]
	(No symbol) [0x0085DEBB]
	(No symbol) [0x0086BFD3]
	(No symbol) [0x0085A0B6]
	(No symbol) [0x00837E08]
	(No symbol) [0x00838F2D]
	GetHandleVerifier [0x00BD8E3A+2540266]
	GetHandleVerifier [0x00C18959+2801161]
	GetHandleVerifier [0x00C1295C+2776588]
	GetHandleVerifier [0x00A02280+612144]
	(No symbol) [0x00914F6C]
	(No symbol) [0x009111D8]
	(No symbol) [0x009112BB]
	(No symbol) [0x00904857]
	BaseThreadInitThunk [0x75D600C9+25]
	RtlGetAppContainerNamedObjectPath [0x77377B4E+286]
	RtlGetAppContainerNamedObjectPath [0x77377B1E+238]


In [78]:
#Result
df_result = pd.DataFrame(list(zip(link, title, username, likes, views, comment, share)), columns=['link', 'title', 'username', 'likes', 'views', 'comment', 'share'])

In [5]:
# Create a DataFrame
df_result = pd.DataFrame({'Links': links, 'Titles': title, 'Username': username, 'Likes': likes, 'Views': views, 'Comment': comment, 'Share': share})
df_result

ValueError: All arrays must be of the same length

In [6]:
print(links)
print(title)
print(username)
print(likes)
print(views)
print(comment)
print(share)

['https://www.tiktok.com/@dndalstri._/video/7233191381664140571', 'https://www.tiktok.com/@melvi11748/video/7222857827184921862', 'https://www.tiktok.com/@animal009funny/video/7219623495574048046', 'https://www.tiktok.com/@catheezkw/video/7222309923462319386', 'https://www.tiktok.com/@androidnyel105/video/7213563146978512154', 'https://www.tiktok.com/@kinosgina/video/7232834547103173893', 'https://www.tiktok.com/@katababa.real/video/7209134009895472411', 'https://www.tiktok.com/@vilmeijuga/video/7228988618163997979', 'https://www.tiktok.com/@ceritarumah_/video/7205460676901391643']
['skip', '#oppo A78 #new #fypシ ', 'skip', 'iyaa keren 😆 #catheez #ratuauliatp #surya #jegel #talkpod #talkpodnet #catheezrandom #masukberanda #fypシ゚viral ', 'Siapa yang hpnya udah bisa dipasang live wallpaper? 🗿#androidtipsandtrick #androidtricks #androidtips #tutorialsmartphone #tipshandphone #TechReview #fyp #fypシ #xyzbca #KelayaBubblePop #metoosmile #androidreview #android ', '🤔🤔🤔 #kinosgina ', 'skip', 's

In [61]:
#Result
df_result = pd.DataFrame(list(zip(link, title, username, likes, views, comment, share)), columns=['link', 'title', 'username', 'likes', 'views', 'comment', 'share'])
df_result.iloc[0,0]

'h'

In [79]:
#export data to excel
df_result.to_excel('tiktok data scrape result.xlsx', index=False)